## Trabajador experto en conocimiento (RAG)

### Un agente de respuesta a preguntas que es un trabajador experto en conocimiento
### Para ser utilizado por empleados de Insurellm, una empresa de tecnología de seguros
### El agente debe ser preciso y la solución debe ser de bajo costo.

Este proyecto utilizará RAG (Retrieval Augmented Generation) para garantizar que nuestro asistente de preguntas y respuestas tenga una alta precisión.

In [1]:
# imports

import os
import glob
from dotenv import load_dotenv
import gradio as gr

In [2]:
# imports para langchain

from langchain.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import CharacterTextSplitter

In [3]:
# El precio es un factor para nuestra empresa, por eso vamos a utilizar un modelo de bajo costo.

MODEL = "gpt-4o-mini"
db_name = "vector_db"

In [4]:
# Cargar variables de entorno en un archivo llamado .env

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')

In [5]:
# Leer documentos usando los cargadores de LangChain
# Tomar todo lo que está en todas las subcarpetas de nuestra base de conocimiento

folders = glob.glob("knowledge-base/*")

# Con agradecimientos a CG y Jon R, estudiantes del curso, por esta solución necesaria para algunos usuarios.
text_loader_kwargs = {'encoding': 'utf-8'}
# Si esto no funciona, algunos usuarios de Windows podrían necesitar descomentar la siguiente línea.
# text_loader_kwargs={'autodetect_encoding': True}

documents = []
for folder in folders:
    doc_type = os.path.basename(folder)
    loader = DirectoryLoader(folder, glob="**/*.md", loader_cls=TextLoader, loader_kwargs=text_loader_kwargs)
    folder_docs = loader.load()
    for doc in folder_docs:
        doc.metadata["doc_type"] = doc_type
        documents.append(doc)

In [6]:
len(documents)

31

In [9]:
documents[24]

Document(metadata={'source': 'knowledge-base/employees/Maxine Thompson.md', 'doc_type': 'employees'}, page_content='# Registro de RR. HH.\n\n# Maxine Thompson\n\n## Resumen\n- **Fecha de nacimiento:** 15 de enero de 1991\n- **Puesto:** Ingeniero de datos\n- **Ubicación:** Austin, Texas\n\n## Progresión profesional en Insurellm\n- **Enero de 2017 - Octubre de 2018**: **Ingeniero de datos júnior**\n* Maxine se unió a Insurellm como ingeniera de datos júnior, centrándose principalmente en procesos ETL y tareas de integración de datos. Aprendió rápidamente la arquitectura de datos de Insurellm, colaborando con otros miembros del equipo para optimizar los flujos de trabajo de datos.\n- **Noviembre de 2018 - Diciembre de 2020**: **Ingeniero de datos**\n* En su nuevo puesto, Maxine amplió sus responsabilidades para incluir el diseño de modelos de datos integrales y la mejora de las medidas de calidad de los datos. Aunque se destacó en habilidades técnicas, los problemas de comunicación con eq

In [10]:
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(documents)

Created a chunk of size 1252, which is longer than the specified 1000


In [11]:
len(chunks)

137

In [14]:
chunks[7]

Document(metadata={'source': 'knowledge-base/products/Markellm.md', 'doc_type': 'products'}, page_content='## Características\n\n- **Emparejamiento impulsado por IA**: Markellm utiliza sofisticados algoritmos de IA para emparejar a los consumidores con los productos de seguros más adecuados según sus necesidades y preferencias individuales. Esto garantiza que ambas partes obtengan las mejores opciones posibles.\n\n- **Interfaz fácil de usar**: Diseñado teniendo en cuenta la experiencia del usuario, Markellm presenta una interfaz intuitiva que permite a los consumidores explorar y comparar fácilmente varias ofertas de seguros de múltiples proveedores.\n\n- **Cotizaciones en tiempo real**: los consumidores pueden recibir cotizaciones en tiempo real de diferentes compañías de seguros, lo que les permite tomar decisiones informadas rápidamente sin una comunicación interminable.')

In [15]:
doc_types = set(chunk.metadata['doc_type'] for chunk in chunks)
print(f"Tipos de documentos encontrados: {', '.join(doc_types)}")

Document types found: products, company, employees, contracts


In [18]:
for chunk in chunks:
    if 'CEO' in chunk.page_content:
        print(chunk)
        print("_________")

page_content='## Soporte

1. **Atención al cliente**: Velocity Auto Solutions tendrá acceso al equipo de atención al cliente de Insurellm por correo electrónico o chatbot, disponible las 24 horas, los 7 días de la semana.
2. **Mantenimiento técnico**: Insurellm realizará el mantenimiento y las actualizaciones regulares de la plataforma Carllm, y cualquier tiempo de inactividad se comunicará con anticipación.
3. **Capacitación y recursos**: Se brindarán sesiones de capacitación inicial para el personal de Velocity Auto Solutions para garantizar el uso eficaz de la suite Carllm. Los recursos y la documentación regulares estarán disponibles en línea.

---

**Aceptado y acordado:**
**Para Velocity Auto Solutions**
Firma: _____________________
Nombre: John Doe
Cargo: CEO
Fecha: _____________________

**Para Insurellm**
Firma: _____________________
Nombre: Jane Smith
Cargo: VP de ventas
Fecha: _____________________' metadata={'source': 'knowledge-base/contracts/Contract with Velocity Auto So